In [ ]:
"""
Trello Audio Extractor

A Python library for downloading MP4 attachments from Trello cards,
extracting WAV audio files, and uploading them back to the cards.
"""

import requests
import os
import json
import subprocess
import tempfile
from typing import Dict, List, Optional
from dotenv import load_dotenv
import logging
from pathlib import Path
import shutil


class TrelloAudioExtractor:
    """
    A class to download MP4 attachments from Trello cards, extract WAV audio,
    and upload the audio files back to the cards.
    
    This class provides functionality to:
    - Fetch all cards from a Trello list
    - Download MP4 attachments from cards
    - Extract WAV audio using ffmpeg
    - Upload WAV files back to the source cards
    - Move successfully processed cards to another list
    """
    
    def __init__(self, download_path: str = "./audio_extraction", 
                 keep_local_files: bool = False,
                 destination_list_id: Optional[str] = None):
        """
        Initialize the Trello Audio Extractor.
        
        Automatically loads credentials from .env file using:
        - TRELLO_API_KEY
        - TRELLO_API_TOKEN
        
        Args:
            download_path (str): Directory to save downloaded and extracted files
            keep_local_files (bool): Whether to keep local files after upload
            destination_list_id (Optional[str]): List ID to move successfully processed cards to
        """
        # Load environment variables from .env file
        load_dotenv()
        
        self.api_key = os.getenv("TRELLO_API_KEY")
        self.token = os.getenv("TRELLO_API_TOKEN")
        
        if not self.api_key:
            raise ValueError("TRELLO_API_KEY not found in environment variables")
        if not self.token:
            raise ValueError("TRELLO_API_TOKEN not found in environment variables")
            
        self.base_url = "https://api.trello.com/1"
        self.download_path = download_path
        self.keep_local_files = keep_local_files
        self.destination_list_id = destination_list_id
        
        # Create download directory if it doesn't exist
        os.makedirs(self.download_path, exist_ok=True)
        
        # Setup logging
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)
        
        # Check if ffmpeg is available
        self._check_ffmpeg()
    
    def _check_ffmpeg(self):
        """Check if ffmpeg is installed and available."""
        try:
            subprocess.run(['ffmpeg', '-version'], 
                         capture_output=True, check=True)
            self.logger.info("ffmpeg found and ready")
        except (subprocess.CalledProcessError, FileNotFoundError):
            raise RuntimeError(
                "ffmpeg not found. Please install ffmpeg:\n"
                "- Windows: Download from https://ffmpeg.org/download.html\n"
                "- macOS: brew install ffmpeg\n"
                "- Linux: sudo apt install ffmpeg (Ubuntu/Debian) or sudo yum install ffmpeg (RHEL/CentOS)"
            )
    
    def get_cards_from_list(self, list_id: str) -> List[Dict]:
        """
        Fetch all cards from the specified Trello list.
        
        Args:
            list_id (str): The ID of the Trello list
            
        Returns:
            List[Dict]: List of card data dictionaries
            
        Raises:
            requests.RequestException: If the API request fails
        """
        url = f"{self.base_url}/lists/{list_id}/cards"
        params = {
            'key': self.api_key,
            'token': self.token,
            'fields': 'id,name,desc,shortUrl',
            'attachments': 'true'
        }
        
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            cards = response.json()
            self.logger.info(f"Fetched {len(cards)} cards from list {list_id}")
            return cards
        except requests.RequestException as e:
            self.logger.error(f"Error fetching cards: {e}")
            raise
    
    def get_mp4_attachments(self, card: Dict) -> List[Dict]:
        """
        Get all MP4 attachments from a card.
        
        Args:
            card (Dict): Card data dictionary
            
        Returns:
            List[Dict]: List of MP4 attachment dictionaries
        """
        mp4_attachments = []
        
        if 'attachments' in card:
            for attachment in card['attachments']:
                # Check if attachment is an MP4 file
                if (attachment.get('name', '').lower().endswith('.mp4') or 
                    attachment.get('mimeType', '').startswith('video/mp4')):
                    mp4_attachments.append(attachment)
        
        return mp4_attachments
    
    def download_attachment(self, attachment: Dict, download_dir: str) -> Dict:
        """
        Download an attachment from Trello.
        
        Args:
            attachment (Dict): Attachment data dictionary
            download_dir (str): Directory to save the downloaded file
            
        Returns:
            Dict: Download result with status and file path
        """
        try:
            # Try different URL approaches for Trello attachments
            attachment_url = attachment['url']
            filename = attachment['name']
            file_path = os.path.join(download_dir, filename)
            
            self.logger.info(f"Downloading attachment: {filename}")
            
            # Method 1: Try the direct URL first (some attachments are publicly accessible)
            try:
                response = requests.get(attachment_url, stream=True, timeout=30)
                if response.status_code == 200:
                    with open(file_path, 'wb') as f:
                        for chunk in response.iter_content(chunk_size=8192):
                            f.write(chunk)
                    
                    file_size = os.path.getsize(file_path)
                    self.logger.info(f"Downloaded {filename} ({file_size} bytes) via direct URL")
                    
                    return {
                        'status': 'success',
                        'file_path': file_path,
                        'filename': filename,
                        'file_size': file_size,
                        'attachment_id': attachment['id']
                    }
            except:
                pass  # Try next method
            
            # Method 2: Use the card attachments API endpoint
            card_id = attachment.get('idCard')
            if card_id:
                api_download_url = f"{self.base_url}/cards/{card_id}/attachments/{attachment['id']}/download"
                
                params = {
                    'key': self.api_key,
                    'token': self.token
                }
                
                response = requests.get(api_download_url, params=params, stream=True, timeout=30)
                if response.status_code == 200:
                    with open(file_path, 'wb') as f:
                        for chunk in response.iter_content(chunk_size=8192):
                            f.write(chunk)
                    
                    file_size = os.path.getsize(file_path)
                    self.logger.info(f"Downloaded {filename} ({file_size} bytes) via API endpoint")
                    
                    return {
                        'status': 'success',
                        'file_path': file_path,
                        'filename': filename,
                        'file_size': file_size,
                        'attachment_id': attachment['id']
                    }
            
            # Method 3: Try with authentication headers
            headers = {
                'Authorization': f'OAuth oauth_consumer_key="{self.api_key}", oauth_token="{self.token}"'
            }
            
            response = requests.get(attachment_url, headers=headers, stream=True, timeout=30)
            if response.status_code == 200:
                with open(file_path, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                
                file_size = os.path.getsize(file_path)
                self.logger.info(f"Downloaded {filename} ({file_size} bytes) via OAuth headers")
                
                return {
                    'status': 'success',
                    'file_path': file_path,
                    'filename': filename,
                    'file_size': file_size,
                    'attachment_id': attachment['id']
                }
            
            # If all methods fail
            return {
                'status': 'error',
                'error': f'Failed to download after trying multiple methods. Last status: {response.status_code}',
                'attachment_id': attachment['id'],
                'filename': filename
            }
            
        except Exception as e:
            self.logger.error(f"Error downloading attachment {attachment.get('name', 'unknown')}: {str(e)}")
            return {
                'status': 'error',
                'error': str(e),
                'attachment_id': attachment.get('id', 'unknown'),
                'filename': attachment.get('name', 'unknown')
            }
    
    def extract_audio_to_wav(self, mp4_path: str, output_dir: str) -> Dict:
        """
        Extract WAV audio from an MP4 file using ffmpeg.
        
        Args:
            mp4_path (str): Path to the input MP4 file
            output_dir (str): Directory to save the extracted WAV file
            
        Returns:
            Dict: Extraction result with status and file path
        """
        try:
            # Generate output WAV filename
            mp4_filename = os.path.basename(mp4_path)
            wav_filename = os.path.splitext(mp4_filename)[0] + '.wav'
            wav_path = os.path.join(output_dir, wav_filename)
            
            self.logger.info(f"Extracting audio from {mp4_filename} to {wav_filename}")
            
            # ffmpeg command to extract audio
            cmd = [
                'ffmpeg',
                '-i', mp4_path,           # Input file
                '-vn',                    # No video
                '-acodec', 'pcm_s16le',  # WAV codec
                '-ar', '44100',          # Sample rate
                '-ac', '2',              # Stereo
                '-y',                    # Overwrite output file
                wav_path                 # Output file
            ]
            
            # Run ffmpeg
            result = subprocess.run(
                cmd,
                capture_output=True,
                text=True,
                check=True
            )
            
            if os.path.exists(wav_path):
                file_size = os.path.getsize(wav_path)
                self.logger.info(f"Successfully extracted audio to {wav_filename} ({file_size} bytes)")
                
                return {
                    'status': 'success',
                    'wav_path': wav_path,
                    'wav_filename': wav_filename,
                    'file_size': file_size,
                    'source_mp4': mp4_path
                }
            else:
                return {
                    'status': 'error',
                    'error': 'WAV file was not created',
                    'source_mp4': mp4_path
                }
                
        except subprocess.CalledProcessError as e:
            self.logger.error(f"ffmpeg error: {e.stderr}")
            return {
                'status': 'error',
                'error': f'ffmpeg failed: {e.stderr}',
                'source_mp4': mp4_path
            }
        except Exception as e:
            self.logger.error(f"Error extracting audio from {mp4_path}: {str(e)}")
            return {
                'status': 'error',
                'error': str(e),
                'source_mp4': mp4_path
            }
    
    def upload_file_to_card(self, card_id: str, file_path: str, file_name: str = None) -> Dict:
        """
        Upload a file as an attachment to a Trello card.
        
        Args:
            card_id (str): The ID of the Trello card
            file_path (str): Path to the file to upload
            file_name (str): Optional custom name for the attachment
            
        Returns:
            Dict: Upload result with status and details
        """
        if not os.path.exists(file_path):
            return {
                'status': 'error',
                'error': f'File not found: {file_path}',
                'file_path': file_path
            }
        
        url = f"{self.base_url}/cards/{card_id}/attachments"
        
        # Get file size for validation
        file_size = os.path.getsize(file_path)
        max_size = 250 * 1024 * 1024  # 250MB limit for Trello
        
        if file_size > max_size:
            return {
                'status': 'error',
                'error': f'File too large: {file_size} bytes (max: {max_size} bytes)',
                'file_path': file_path,
                'file_size': file_size
            }
        
        try:
            # Use the original filename if no custom name provided
            if not file_name:
                file_name = os.path.basename(file_path)
            
            with open(file_path, 'rb') as file:
                files = {
                    'file': (file_name, file, 'audio/wav')
                }
                data = {
                    'key': self.api_key,
                    'token': self.token,
                    'name': file_name
                }
                
                self.logger.info(f"Uploading {file_name} ({file_size} bytes) to card {card_id}")
                
                response = requests.post(url, files=files, data=data)
                response.raise_for_status()
                
                attachment_info = response.json()
                
                return {
                    'status': 'success',
                    'attachment_id': attachment_info['id'],
                    'attachment_url': attachment_info['url'],
                    'file_name': file_name,
                    'file_path': file_path,
                    'file_size': file_size
                }
                
        except requests.RequestException as e:
            return {
                'status': 'error',
                'error': f'Upload failed: {str(e)}',
                'file_path': file_path
            }
        except Exception as e:
            return {
                'status': 'error',
                'error': f'Unexpected error: {str(e)}',
                'file_path': file_path
            }
    
    def move_card_to_list(self, card_id: str, destination_list_id: str) -> Dict:
        """
        Move a card to another list.
        
        Args:
            card_id (str): The ID of the card to move
            destination_list_id (str): The ID of the destination list
            
        Returns:
            Dict: Move result with status and details
        """
        url = f"{self.base_url}/cards/{card_id}"
        params = {
            'key': self.api_key,
            'token': self.token
        }
        data = {
            'idList': destination_list_id
        }
        
        try:
            response = requests.put(url, params=params, data=data)
            response.raise_for_status()
            
            self.logger.info(f"Successfully moved card {card_id} to list {destination_list_id}")
            
            return {
                'status': 'success',
                'card_id': card_id,
                'destination_list_id': destination_list_id
            }
            
        except requests.RequestException as e:
            self.logger.error(f"Failed to move card {card_id}: {str(e)}")
            return {
                'status': 'error',
                'card_id': card_id,
                'destination_list_id': destination_list_id,
                'error': str(e)
            }
    
    def process_card(self, card: Dict) -> Dict:
        """
        Process a single card: download MP4s, extract WAV audio, upload WAV files.
        
        Args:
            card (Dict): Card information
            
        Returns:
            Dict: Processing results for this card
        """
        card_results = {
            'card_info': card,
            'mp4_attachments': [],
            'downloads': [],
            'extractions': [],
            'uploads': [],
            'move_result': None,
            'processing_successful': False,
            'errors': []
        }
        
        try:
            # Create subdirectory for this card
            card_name = self.sanitize_filename(card['name'])
            card_dir = os.path.join(self.download_path, f"card_{card['id']}_{card_name}")
            os.makedirs(card_dir, exist_ok=True)
            
            # Get MP4 attachments
            mp4_attachments = self.get_mp4_attachments(card)
            card_results['mp4_attachments'] = mp4_attachments
            
            if not mp4_attachments:
                self.logger.info(f"No MP4 attachments found in card: {card['name']}")
                return card_results
            
            self.logger.info(f"Found {len(mp4_attachments)} MP4 attachments in card: {card['name']}")
            
            successful_extractions = 0
            
            # Process each MP4 attachment
            for attachment in mp4_attachments:
                # Download MP4
                download_result = self.download_attachment(attachment, card_dir)
                card_results['downloads'].append(download_result)
                
                if download_result['status'] == 'success':
                    # Extract WAV audio
                    extraction_result = self.extract_audio_to_wav(
                        download_result['file_path'], 
                        card_dir
                    )
                    extraction_result['source_attachment_id'] = attachment['id']
                    card_results['extractions'].append(extraction_result)
                    
                    if extraction_result['status'] == 'success':
                        # Upload WAV to card
                        upload_result = self.upload_file_to_card(
                            card['id'],
                            extraction_result['wav_path'],
                            f"Audio: {extraction_result['wav_filename']}"
                        )
                        upload_result['source_attachment_id'] = attachment['id']
                        upload_result['source_filename'] = attachment['name']
                        card_results['uploads'].append(upload_result)
                        
                        if upload_result['status'] == 'success':
                            successful_extractions += 1
                            self.logger.info(f"Successfully processed {attachment['name']} -> {extraction_result['wav_filename']}")
                            
                            # Clean up files if not keeping them
                            if not self.keep_local_files:
                                try:
                                    os.remove(download_result['file_path'])  # Remove MP4
                                    os.remove(extraction_result['wav_path'])  # Remove WAV
                                except Exception as e:
                                    self.logger.warning(f"Could not delete files: {e}")
                        else:
                            self.logger.error(f"Failed to upload WAV: {upload_result['error']}")
                    else:
                        self.logger.error(f"Failed to extract audio: {extraction_result['error']}")
                else:
                    self.logger.error(f"Failed to download MP4: {download_result['error']}")
            
            # Determine if processing was successful
            if successful_extractions > 0:
                card_results['processing_successful'] = True
                self.logger.info(f"Card {card['name']} processed successfully with {successful_extractions} audio extractions")
                
                # Move card to destination list if specified
                if self.destination_list_id:
                    move_result = self.move_card_to_list(card['id'], self.destination_list_id)
                    card_results['move_result'] = move_result
                    
                    if move_result['status'] == 'success':
                        self.logger.info(f"Card {card['name']} moved to destination list")
                    else:
                        self.logger.error(f"Failed to move card {card['name']}: {move_result['error']}")
            else:
                self.logger.warning(f"Card {card['name']} had MP4 attachments but no successful audio extractions")
            
            # Clean up empty directory if not keeping files
            if not self.keep_local_files and not os.listdir(card_dir):
                os.rmdir(card_dir)
                
        except Exception as e:
            error_info = {
                'error': str(e),
                'context': 'card_processing'
            }
            card_results['errors'].append(error_info)
            self.logger.error(f"Error processing card {card['name']}: {str(e)}")
        
        return card_results
    
    def sanitize_filename(self, filename: str) -> str:
        """
        Sanitize filename by removing/replacing invalid characters.
        
        Args:
            filename (str): Original filename
            
        Returns:
            str: Sanitized filename safe for filesystem
        """
        # Remove or replace invalid characters
        invalid_chars = '<>:"/\\|?*'
        for char in invalid_chars:
            filename = filename.replace(char, '_')
        
        # Limit length and strip whitespace
        filename = filename.strip()[:200]
        
        return filename
    
    def process_list(self, list_id: str) -> Dict:
        """
        Master function to process all cards in a list: download MP4s, extract WAV audio, 
        upload WAV files, and optionally move successful cards to another list.
        
        Args:
            list_id (str): The ID of the Trello list to process
            
        Returns:
            Dict: Summary of the processing results
        """
        self.logger.info(f"Starting to process Trello list: {list_id}")
        if self.destination_list_id:
            self.logger.info(f"Will move successful cards to list: {self.destination_list_id}")
        
        results = {
            'list_id': list_id,
            'destination_list_id': self.destination_list_id,
            'total_cards': 0,
            'cards_with_mp4': 0,
            'cards_processed_successfully': 0,
            'cards_moved_successfully': 0,
            'total_mp4_attachments': 0,
            'total_audio_extractions': 0,
            'total_wav_uploads': 0,
            'successful_downloads': [],
            'failed_downloads': [],
            'successful_extractions': [],
            'failed_extractions': [],
            'successful_uploads': [],
            'failed_uploads': [],
            'successful_moves': [],
            'failed_moves': [],
            'card_results': [],
            'processing_errors': []
        }
        
        try:
            # Get all cards from the list
            cards = self.get_cards_from_list(list_id)
            results['total_cards'] = len(cards)
            
            for card in cards:
                self.logger.info(f"Processing card: {card['name']} (ID: {card['id']})")
                
                card_results = self.process_card(card)
                results['card_results'].append(card_results)
                
                # Update summary statistics
                if card_results['mp4_attachments']:
                    results['cards_with_mp4'] += 1
                    results['total_mp4_attachments'] += len(card_results['mp4_attachments'])
                
                # Track successful processing
                if card_results['processing_successful']:
                    results['cards_processed_successfully'] += 1
                
                # Process downloads
                for download in card_results['downloads']:
                    if download['status'] == 'success':
                        results['successful_downloads'].append(download)
                    else:
                        results['failed_downloads'].append(download)
                
                # Process extractions
                for extraction in card_results['extractions']:
                    if extraction['status'] == 'success':
                        results['successful_extractions'].append(extraction)
                        results['total_audio_extractions'] += 1
                    else:
                        results['failed_extractions'].append(extraction)
                
                # Process uploads
                for upload in card_results['uploads']:
                    if upload['status'] == 'success':
                        results['successful_uploads'].append(upload)
                        results['total_wav_uploads'] += 1
                    else:
                        results['failed_uploads'].append(upload)
                
                # Process move result
                if card_results['move_result']:
                    if card_results['move_result']['status'] == 'success':
                        results['successful_moves'].append(card_results['move_result'])
                        results['cards_moved_successfully'] += 1
                    else:
                        results['failed_moves'].append(card_results['move_result'])
                
                # Process errors
                for error in card_results['errors']:
                    error['card_name'] = card['name']
                    error['card_id'] = card['id']
                    results['processing_errors'].append(error)
            
            # Clean up empty directories if not keeping local files
            if not self.keep_local_files:
                self.cleanup_empty_directories()
            
            # Save results summary
            self.save_results_summary(list_id, results)
            
        except Exception as e:
            self.logger.error(f"Fatal error processing list {list_id}: {str(e)}")
            results['processing_errors'].append({
                'error': f"Fatal error: {str(e)}",
                'context': 'list_processing'
            })
        
        return results
    
    def cleanup_empty_directories(self):
        """Remove empty directories in the download path."""
        try:
            for root, dirs, files in os.walk(self.download_path, topdown=False):
                for directory in dirs:
                    dir_path = os.path.join(root, directory)
                    try:
                        if not os.listdir(dir_path):  # Directory is empty
                            os.rmdir(dir_path)
                            self.logger.info(f"Removed empty directory: {dir_path}")
                    except OSError:
                        pass  # Directory not empty or other error
        except Exception as e:
            self.logger.warning(f"Error during cleanup: {e}")
    
    def save_results_summary(self, list_id: str, results: Dict):
        """
        Save processing results to a JSON file.
        
        Args:
            list_id (str): The processed list ID
            results (Dict): Processing results
        """
        summary_file = os.path.join(self.download_path, f"audio_extraction_summary_{list_id}.json")
        
        try:
            # Create a serializable copy of results
            serializable_results = json.loads(json.dumps(results, default=str))
            
            with open(summary_file, 'w', encoding='utf-8') as f:
                json.dump(serializable_results, f, indent=2, ensure_ascii=False)
            self.logger.info(f"Results summary saved to: {summary_file}")
        except Exception as e:
            self.logger.error(f"Error saving results summary: {str(e)}")
    
    def print_summary(self, results: Dict):
        """
        Print a formatted summary of the processing results.
        
        Args:
            results (Dict): Processing results to summarize
        """
        print("\n" + "="*70)
        print("TRELLO AUDIO EXTRACTOR - SUMMARY")
        print("="*70)
        print(f"List ID: {results['list_id']}")
        if results.get('destination_list_id'):
            print(f"Destination List ID: {results['destination_list_id']}")
        print(f"Total cards processed: {results['total_cards']}")
        print(f"Cards with MP4 attachments: {results['cards_with_mp4']}")
        print(f"Cards processed successfully: {results['cards_processed_successfully']}")
        if results.get('destination_list_id'):
            print(f"Cards moved successfully: {results['cards_moved_successfully']}")
        print(f"Total MP4 attachments found: {results['total_mp4_attachments']}")
        print(f"Audio extractions completed: {results['total_audio_extractions']}")
        print(f"WAV files uploaded to cards: {results['total_wav_uploads']}")
        print(f"Failed downloads: {len(results['failed_downloads'])}")
        print(f"Failed extractions: {len(results['failed_extractions'])}")
        print(f"Failed uploads: {len(results['failed_uploads'])}")
        if results.get('failed_moves'):
            print(f"Failed card moves: {len(results['failed_moves'])}")
        print(f"Processing errors: {len(results['processing_errors'])}")
        
        if results['successful_uploads']:
            print(f"\n✅ SUCCESSFUL WAV UPLOADS ({len(results['successful_uploads'])}):")
            for upload in results['successful_uploads']:
                print(f"  🎵 {upload['file_name']}")
                print(f"     Source: {upload.get('source_filename', 'Unknown MP4')}")
                print(f"     Size: {upload.get('file_size', 0)} bytes")
                print(f"     Attachment URL: {upload['attachment_url']}")
                print()
        
        if results['failed_extractions']:
            print(f"\n❌ FAILED AUDIO EXTRACTIONS ({len(results['failed_extractions'])}):")
            for extraction in results['failed_extractions']:
                print(f"  🎬 {os.path.basename(extraction['source_mp4'])}")
                print(f"     Error: {extraction['error']}")
                print()
        
        if results['failed_uploads']:
            print(f"\n⚠️  FAILED UPLOADS ({len(results['failed_uploads'])}):")
            for upload in results['failed_uploads']:
                print(f"  📁 {upload.get('file_name', 'Unknown file')}")
                print(f"     Error: {upload['error']}")
                print()
        
        if results.get('successful_moves'):
            print(f"\n✅ SUCCESSFUL CARD MOVES ({len(results['successful_moves'])}):")
            for move in results['successful_moves']:
                # Find the card name from card_results
                card_name = "Unknown Card"
                for card_result in results['card_results']:
                    if card_result['card_info']['id'] == move['card_id']:
                        card_name = card_result['card_info']['name']
                        break
                print(f"  📋 {card_name}")
                print(f"     Moved to list: {move['destination_list_id']}")
                print()
        
        if results.get('failed_moves'):
            print(f"\n❌ FAILED CARD MOVES ({len(results['failed_moves'])}):")
            for move in results['failed_moves']:
                # Find the card name from card_results
                card_name = "Unknown Card"
                for card_result in results['card_results']:
                    if card_result['card_info']['id'] == move['card_id']:
                        card_name = card_result['card_info']['name']
                        break
                print(f"  📋 {card_name}")
                print(f"     Target list: {move['destination_list_id']}")
                print(f"     Error: {move['error']}")
                print()
        
        if results['processing_errors']:
            print(f"\n⚠️  PROCESSING ERRORS ({len(results['processing_errors'])}):")
            for error in results['processing_errors']:
                print(f"  📋 {error.get('card_name', 'Unknown card')}")
                print(f"     Error: {error['error']}")
                print()
        
        print("="*70)


def extract_audio_from_trello_cards(list_id: str, 
                                   download_path: str = "./audio_extraction",
                                   keep_local_files: bool = False,
                                   destination_list_id: Optional[str] = None) -> Dict:
    """
    Convenience function to extract audio from MP4 attachments in Trello cards.
    
    Args:
        list_id (str): The ID of the Trello list to process
        download_path (str): Directory to save downloaded and extracted files
        keep_local_files (bool): Whether to keep local files after upload
        destination_list_id (Optional[str]): List ID to move successfully processed cards to
        
    Returns:
        Dict: Processing results summary
    """
    extractor = TrelloAudioExtractor(download_path, keep_local_files, destination_list_id)
    results = extractor.process_list(list_id)
    extractor.print_summary(results)
    return results

In [ ]:
list_id = os.getenv("TRELLO_MP4_LINK_LIST_ID")
destination_list_id = os.getenv("TRELLO_AUDIO_LINK_LIST_ID")

try:
    results = extract_audio_from_trello_cards(
        list_id=list_id,
        download_path="./audio_extraction",
        keep_local_files=False,  # Delete local files after upload
        destination_list_id=destination_list_id  # Move successful cards here
    )
    
    print(f"Processing complete!")
    print(f"Audio extractions: {results['total_audio_extractions']}")
    print(f"WAV uploads: {results['total_wav_uploads']}")
    print(f"Cards moved: {results['cards_moved_successfully']} cards")
    
except Exception as e:
    print(f"Error: {e}")

# Trello Audio Extractor

A Python library for extracting WAV audio from MP4 attachments in Trello cards.

## Requirements

Create a `requirements.txt` file:

```txt
requests>=2.28.0
python-dotenv>=1.0.0
```

**System Requirements:**
- **ffmpeg** must be installed on your system for audio extraction

### Installing ffmpeg:

**Windows:**
- Download from https://ffmpeg.org/download.html
- Extract and add to PATH

**macOS:**
```bash
brew install ffmpeg
```

**Linux (Ubuntu/Debian):**
```bash
sudo apt install ffmpeg
```

**Linux (RHEL/CentOS):**
```bash
sudo yum install ffmpeg
```

## Environment Setup

Create a `.env` file in your project directory:

```env
TRELLO_API_KEY=your_trello_api_key_here
TRELLO_API_TOKEN=your_trello_api_token_here
```

## Installation

```bash
pip install -r requirements.txt
```

## Simple Usage Example

```python
from trello_audio_extractor import extract_audio_from_trello_cards

# Extract audio from MP4 attachments and upload WAV files to cards
list_id = "your_source_list_id_here"
destination_list_id = "your_destination_list_id_here"  # Optional

results = extract_audio_from_trello_cards(
    list_id=list_id,
    destination_list_id=destination_list_id
)
```

## Advanced Usage Example

```python
from trello_audio_extractor import TrelloAudioExtractor

# Create extractor instance with custom settings
extractor = TrelloAudioExtractor(
    download_path="./my_audio_extractions",
    keep_local_files=True,  # Keep downloaded and extracted files
    destination_list_id="your_completed_list_id"  # Move successful cards here
)

# Process a list
results = extractor.process_list("your_source_list_id")

# Print detailed summary
extractor.print_summary(results)

# Access detailed results
print(f"Audio extractions completed: {results['total_audio_extractions']}")
print(f"WAV files uploaded: {results['total_wav_uploads']}")
print(f"Cards moved to destination: {results['cards_moved_successfully']}")
print(f"From {results['cards_with_mp4']} cards with MP4 attachments")
```

## How to Get Your Trello API Credentials

1. Go to https://trello.com/app-key
2. Copy your API Key
3. Click the "Token" link to generate a token
4. Copy the token
5. Add both to your `.env` file

## How to Get Trello List IDs

### For Source and Destination Lists:

1. Open your Trello board in a web browser
2. Go to any list you want to get the ID for
3. Click on a card in that list
4. Look at the URL - it will be like: `https://trello.com/c/CARD_ID/card-name`
5. Add `.json` to the end: `https://trello.com/c/CARD_ID/card-name.json`
6. Open that URL in your browser
7. Look for `"idList"` in the JSON - that's your list ID

### Typical Workflow Setup:
- **Source List**: "Videos with MP4s" (contains cards with MP4 attachments)
- **Destination List**: "Audio Extracted" (where successfully processed cards are moved)

This creates a clear workflow where cards move from "Videos" → "Audio Extracted" after audio is extracted and attached.

## Features

- **Complete Audio Workflow**: Downloads MP4 attachments, extracts WAV audio, uploads back to cards
- **Workflow Management**: Optionally moves successfully processed cards to another list
- **High-Quality Audio**: Extracts WAV audio at 44.1kHz, 16-bit, stereo
- **Batch Processing**: Processes entire lists automatically with multiple MP4s per card
- **Smart File Management**: Organizes downloads and optionally cleans up after upload
- **Progress Tracking**: Detailed logging and summary reports for all operations
- **Error Handling**: Robust error handling with detailed error reporting
- **ffmpeg Integration**: Uses ffmpeg for reliable, high-quality audio extraction
- **Summary Reports**: Generates JSON summary files for each processing run

## Complete Workflow

1. **Scan Trello List**: Fetches all cards from the specified source list
2. **Find MP4 Attachments**: Identifies MP4 video file attachments on cards
3. **Download MP4s**: Downloads MP4 files to local storage
4. **Extract Audio**: Uses ffmpeg to extract high-quality WAV audio
5. **Upload WAV Files**: Attaches extracted WAV files to their source Trello cards
6. **Move Cards**: Optionally moves successfully processed cards to a destination list
7. **Cleanup**: Optionally deletes local files after successful upload
8. **Report**: Generates detailed summary of all operations

## Audio Extraction Details

**Output Format:**
- **Format**: WAV (uncompressed)
- **Sample Rate**: 44.1kHz
- **Bit Depth**: 16-bit
- **Channels**: Stereo (2 channels)
- **Codec**: PCM signed 16-bit little-endian

**ffmpeg Command Used:**
```bash
ffmpeg -i input.mp4 -vn -acodec pcm_s16le -ar 44100 -ac 2 -y output.wav
```

## File Structure (when keep_local_files=True)

```
audio_extraction/
├── card_123_CardName1/
│   ├── video1.mp4 (downloaded from card)
│   ├── video1.wav (extracted audio, also uploaded to card)
│   ├── video2.mp4 (downloaded from card)
│   └── video2.wav (extracted audio, also uploaded to card)
├── card_456_CardName2/
│   ├── movie.mp4 (downloaded from card)
│   └── movie.wav (extracted audio, also uploaded to card)
└── audio_extraction_summary_list123.json
```

**Note**: When `keep_local_files=False` (default), local files are deleted after successful upload to keep storage clean.

## Important Notes

- **File Size Limit**: Trello has a 250MB attachment limit for both MP4s and WAV files
- **ffmpeg Required**: This tool requires ffmpeg to be installed and available in PATH
- **Audio Quality**: WAV files are uncompressed and may be large - monitor storage space
- **Supported Input**: Only processes MP4 video files attached to cards
- **Storage**: By default, local files are deleted after upload to save space (configurable)
- **Error Handling**: Failed downloads, extractions, or uploads are tracked and reported separately

## Troubleshooting

**"ffmpeg not found" Error:**
- Make sure ffmpeg is installed and available in your system PATH
- Test with: `ffmpeg -version` in your terminal

**Large File Warnings:**
- WAV files can be quite large (uncompressed audio)
- Consider storage space when processing many videos
- Enable cleanup (`keep_local_files=False`) to save space

**No MP4 Attachments Found:**
- Ensure your cards have MP4 files as attachments (not just links)
- Check that attachment names end with `.mp4` or have `video/mp4` MIME type

## Example Output

```
TRELLO AUDIO EXTRACTOR - SUMMARY
======================================================================
List ID: 507f1f77bcf86cd799439011
Destination List ID: 507f1f77bcf86cd799439012
Total cards processed: 15
Cards with MP4 attachments: 8
Cards processed successfully: 7
Cards moved successfully: 7
Total MP4 attachments found: 12
Audio extractions completed: 11
WAV files uploaded to cards: 11
Failed downloads: 0
Failed extractions: 1
Failed uploads: 0
Failed card moves: 0
Processing errors: 0

✅ SUCCESSFUL WAV UPLOADS (11):
  🎵 Audio: presentation_video.wav
     Source: presentation_video.mp4
     Size: 15728640 bytes
     Attachment URL: https://trello-attachments.s3.amazonaws.com/...
```

This tool is perfect for extracting audio from video content stored in Trello cards, such as meeting recordings, presentations, or any video content where you need the audio track separately.